# Data preparation

In this module, we will start processing the dataset

In [104]:

import os
import pandas as pd

In [105]:
DATA_FOLDER = '../data'

df_compas = pd.read_csv(os.path.join(DATA_FOLDER, 'compas-scores-two-years.csv'))

## First dataset reduction

We reduce the fields that contain administrative information, the fields that are 'replicated' in other fields (for example age and dob, here we only keep one) and fields that to not contain information that can contribute to prediction (for example description fields)

In [106]:
columns_to_keep = [ "sex", "age", "race", "juv_fel_count", "decile_score",
                   "juv_misd_count", "juv_other_count", "priors_count",
                   "days_b_screening_arrest", "c_jail_in", "c_jail_out",
                   "c_days_from_compas", "c_charge_degree", "v_decile_score",
                   "in_custody", "out_custody", "start", "end", "event", "two_year_recid" ]


df_reduced = df_compas[columns_to_keep].copy()



## Processing

We process fields that can be combined into one through difference

-  "c_jail_in",  "c_jail_out" will become c_jail_time
- "r_jail_in", "r_jail_out" will become r_jail_time
- "in_custody", "out_custody" will become custody_time
- "start", "end" will become end_start

In [107]:
# "c_jail_in",  "c_jail_out" will become c_jail_time
df_reduced['c_jail_in'] = pd.to_datetime(
    df_reduced['c_jail_in'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df_reduced['c_jail_out'] = pd.to_datetime(
    df_reduced['c_jail_out'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df_reduced['c_jail_time'] = abs((df_reduced['c_jail_out'] - df_reduced['c_jail_in']).dt.days)

# "in_custody", "out_custody" will become days_in_custody
df_reduced['in_custody'] = pd.to_datetime(
    df_reduced['in_custody'], format='%Y-%m-%d', errors='coerce')
df_reduced['out_custody'] = pd.to_datetime(
    df_reduced['out_custody'], format='%Y-%m-%d', errors='coerce')
df_reduced['days_in_custody'] = abs((df_reduced['out_custody'] - df_reduced['in_custody']).dt.days)

# "start", "end" will become end_start
df_reduced['end_start'] = abs((df_reduced['end'] - df_reduced['start']))

#remove all the columns that are not needed
df_reduced = df_reduced.drop(['c_jail_in', 'c_jail_out', 
                              'in_custody', 'out_custody',
                              'start', 'end'], axis=1)

# save the dataset
df_reduced.to_csv(os.path.join(DATA_FOLDER, 'df_reduced.csv'), index=False)


In [108]:
# gather information about the dataset - data types and missing values
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7214 entries, 0 to 7213
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sex                      7214 non-null   object 
 1   age                      7214 non-null   int64  
 2   race                     7214 non-null   object 
 3   juv_fel_count            7214 non-null   int64  
 4   decile_score             7214 non-null   int64  
 5   juv_misd_count           7214 non-null   int64  
 6   juv_other_count          7214 non-null   int64  
 7   priors_count             7214 non-null   int64  
 8   days_b_screening_arrest  6907 non-null   float64
 9   c_days_from_compas       7192 non-null   float64
 10  c_charge_degree          7214 non-null   object 
 11  v_decile_score           7214 non-null   int64  
 12  event                    7214 non-null   int64  
 13  two_year_recid           7214 non-null   int64  
 14  c_jail_time             

In [109]:
# knn imputation of missing values in days_b_screening_arrest, c_jail_time, days_in_custody

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
df_reduced['days_b_screening_arrest'] = imputer.fit_transform(df_reduced[['days_b_screening_arrest']])
df_reduced['c_jail_time'] = imputer.fit_transform(df_reduced[['c_jail_time']])
df_reduced['days_in_custody'] = imputer.fit_transform(df_reduced[['days_in_custody']])

df_reduced.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7214 entries, 0 to 7213
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sex                      7214 non-null   object 
 1   age                      7214 non-null   int64  
 2   race                     7214 non-null   object 
 3   juv_fel_count            7214 non-null   int64  
 4   decile_score             7214 non-null   int64  
 5   juv_misd_count           7214 non-null   int64  
 6   juv_other_count          7214 non-null   int64  
 7   priors_count             7214 non-null   int64  
 8   days_b_screening_arrest  7214 non-null   float64
 9   c_days_from_compas       7192 non-null   float64
 10  c_charge_degree          7214 non-null   object 
 11  v_decile_score           7214 non-null   int64  
 12  event                    7214 non-null   int64  
 13  two_year_recid           7214 non-null   int64  
 14  c_jail_time             

In [110]:
# encoding of categorical features

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

# transform feature 'sex'
df_reduced['sex'] = df_reduced['sex'].astype('category')
ohe_sex = pd.DataFrame(ohe.fit_transform(df_reduced[['sex']]).toarray(), columns=ohe.get_feature_names_out(['sex']))
df_reduced = pd.concat([df_reduced, ohe_sex], axis=1)

# transform feature 'race'
df_reduced['race'] = df_reduced['race'].astype('category')
ohe_race = pd.DataFrame(ohe.fit_transform(df_reduced[['race']]).toarray(), columns=ohe.get_feature_names_out(['race']))
df_reduced = pd.concat([df_reduced, ohe_race], axis=1)

# transform feature 'c_charge_degree'
df_reduced['c_charge_degree'] = df_reduced['c_charge_degree'].astype('category')
ohe_c_charge_degree = pd.DataFrame(ohe.fit_transform(df_reduced[['c_charge_degree']]).toarray(), columns=ohe.get_feature_names_out(['c_charge_degree']))
df_reduced = pd.concat([df_reduced, ohe_c_charge_degree], axis=1)

# remove original categorical features
df_reduced = df_reduced.drop(['sex', 'race', 'c_charge_degree'], axis=1)

df_reduced.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7214 entries, 0 to 7213
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      7214 non-null   int64  
 1   juv_fel_count            7214 non-null   int64  
 2   decile_score             7214 non-null   int64  
 3   juv_misd_count           7214 non-null   int64  
 4   juv_other_count          7214 non-null   int64  
 5   priors_count             7214 non-null   int64  
 6   days_b_screening_arrest  7214 non-null   float64
 7   c_days_from_compas       7192 non-null   float64
 8   v_decile_score           7214 non-null   int64  
 9   event                    7214 non-null   int64  
 10  two_year_recid           7214 non-null   int64  
 11  c_jail_time              7214 non-null   float64
 12  days_in_custody          7214 non-null   float64
 13  end_start                7214 non-null   int64  
 14  sex_Female              

In [111]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7214 entries, 0 to 7213
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      7214 non-null   int64  
 1   juv_fel_count            7214 non-null   int64  
 2   decile_score             7214 non-null   int64  
 3   juv_misd_count           7214 non-null   int64  
 4   juv_other_count          7214 non-null   int64  
 5   priors_count             7214 non-null   int64  
 6   days_b_screening_arrest  7214 non-null   float64
 7   c_days_from_compas       7192 non-null   float64
 8   v_decile_score           7214 non-null   int64  
 9   event                    7214 non-null   int64  
 10  two_year_recid           7214 non-null   int64  
 11  c_jail_time              7214 non-null   float64
 12  days_in_custody          7214 non-null   float64
 13  end_start                7214 non-null   int64  
 14  sex_Female              

In [112]:
labels = ['two_year_recid', 'decile_score', 'v_decile_score']

df_features = df_reduced.copy().drop(labels, axis=1)

from sklearn.preprocessing import StandardScaler

features_scaled = StandardScaler().fit(df_features).transform(df_features)

pd.DataFrame(features_scaled, columns=df_features.columns).describe()


,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,event,c_jail_time,days_in_custody,...,sex_Female,sex_Male,race_African-American,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other,c_charge_degree_F,c_charge_degree_M
count,7.214000e+03,7.214000e+03,7.214000e+03,7.214000e+03,7.214000e+03,7.214000e+03,7.192000e+03,7.214000e+03,7.214000e+03,7.214000e+03,...,7.214000e+03,7.214000e+03,7.214000e+03,7.214000e+03,7.214000e+03,7.214000e+03,7.214000e+03,7.214000e+03,7.214000e+03,7.214000e+03
mean,-4.530769e-17,2.807107e-17,3.841304e-17,2.216137e-17,5.170986e-17,3.939799e-18,-1.234953e-18,1.526672e-17,-3.447324e-17,-1.231187e-18,...,6.697658e-17,-1.418328e-16,5.220234e-17,2.265384e-17,-8.987666e-17,-2.659364e-17,1.280435e-17,-1.181940e-17,-1.009573e-16,-1.083445e-17
std,1.000069e+00,1.000069e+00,1.000069e+00,1.000069e+00,1.000069e+00,1.000069e+00,1.000070e+00,1.000069e+00,1.000069e+00,1.000069e+00,...,1.000069e+00,1.000069e+00,1.000069e+00,1.000069e+00,1.000069e+00,1.000069e+00,1.000069e+00,1.000069e+00,1.000069e+00,1.000069e+00
min,-1.414692e+00,-1.418546e-01,-1.874142e-01,-2.180649e-01,-7.112398e-01,-5.626061e+00,-1.750935e-01,-7.876518e-01,-3.661522e-01,-2.350366e-01,...,-4.896243e-01,-2.042382e+00,-1.024986e+00,-6.675016e-02,-7.180155e-01,-3.112116e-01,-5.001389e-02,-2.348215e-01,-1.353233e+00,-7.389709e-01
25%,-8.258674e-01,-1.418546e-01,-1.874142e-01,-2.180649e-01,-7.112398e-01,-5.803639e-02,-1.720606e-01,-7.876518e-01,-3.661522e-01,-2.298454e-01,...,-4.896243e-01,4.896243e-01,-1.024986e+00,-6.675016e-02,-7.180155e-01,-3.112116e-01,-5.001389e-02,-2.348215e-01,-1.353233e+00,-7.389709e-01
50%,-3.211610e-01,-1.418546e-01,-1.874142e-01,-2.180649e-01,-3.015884e-01,-5.803639e-02,-1.720606e-01,-7.876518e-01,-3.461385e-01,-2.246541e-01,...,-4.896243e-01,4.896243e-01,9.756228e-01,-6.675016e-02,-7.180155e-01,-3.112116e-01,-5.001389e-02,-2.348215e-01,7.389709e-01,-7.389709e-01
75%,6.041343e-01,-1.418546e-01,-1.874142e-01,-2.180649e-01,3.128887e-01,-4.455449e-02,-1.690277e-01,1.269597e+00,-6.594646e-02,-7.929927e-02,...,-4.896243e-01,4.896243e-01,9.756228e-01,-6.675016e-02,1.392728e+00,-3.112116e-01,-5.001389e-02,-2.348215e-01,7.389709e-01,1.353233e+00
max,5.146493e+00,4.205770e+01,2.660539e+01,3.367679e+01,7.072137e+00,1.420581e+01,2.859198e+01,1.269597e+00,1.562481e+01,3.109412e+01,...,2.042382e+00,4.896243e-01,9.756228e-01,1.498124e+01,1.392728e+00,3.213248e+00,1.999444e+01,4.258554e+00,7.389709e-01,1.353233e+00
